# Financial Fraud Detection

- The objective of this notebook is to showcase the usage of the [___financial-fraud-training___ container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/cugraph/containers/financial-fraud-training) and how to deploy the produced trained models on [NVIDIA Dynamo-Triton](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver).
- We use [IBM TabFormer](https://github.com/IBM/TabFormer) as an example dataset and the dataset is preprocess before model training

NOTE:
* The preprocessing code is written specifically for the TabFormer dataset and will not work with other datasets.
* Additionally, a familiarity with [Jupyter](https://docs.jupyter.org/en/latest/what_is_jupyter.html) is assumed.

# Environment Setup (Sagemaker studio)
This Notebook is designed to work in a Sagemaker studio notebook and we have to run the following commands to make our conda environment function within Sagemaker

Please create a Conda environment and add that to the notebook - See the [README](../README.md) file

In [ ]:
import subprocess
import sys
from IPython.display import clear_output

def execute_with_progress(command):
    cmd = f"conda install -y {package}"
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, 
                              stderr=subprocess.STDOUT, universal_newlines=True)
    
    lines = []
    for line in process.stdout:
        lines.append(line.strip())
        
        # Show only relevant lines
        if any(keyword in line.lower() for keyword in ['installing', 'downloading', 'extracting', 'complete']):
            clear_output(wait=True)
            print(f"Status: {line.strip()}")
    
    return process.wait()


In [ ]:
execute_with_progress("conda env create -f ~/sample-financial-fraud-detection-with-nvidia/conda/notebook_env.yaml")

In [ ]:
execute_with_progress("conda install -y ipykernel --name fraud_blueprint_env")

In [ ]:
execute_with_progress("python -m ipykernel install --user --name fraud_blueprint_env --display-name 'fraud_blueprint_env'")

### Activate the env in the kernel

Now choose the `fraud_blueprint_env` kernel from within jupyterlab for this notebook

-----
## Import libraries

In [ ]:
import os
import sys
import json
import time
import boto3

## Prepare the AWS Environment variables from the infrastructure stack

In [ ]:
ssm_client = boto3.client('ssm')
cfn_client = boto3.client('cloudformation')
elb_client = boto3.client('elb')

def get_cfn_output(stack_name, output_key):
    response = cfn_client.describe_stacks(StackName=stack_name)
    outputs = response['Stacks'][0]['Outputs']
    for output in outputs:
        if output['OutputKey'] == output_key:
            return output['OutputValue']
    return None

def get_inference_host():
    response = elb_client.describe_load_balancers()['LoadBalancerDescriptions']

    for elb in response:
        if elb['LoadBalancerName'].startswith('k8s-triton'):
            return elb['DNSName']

bucket_name = get_cfn_output("NvidiaFraudDetectionBlueprintModelExtractor", "SourceBucketName")

sagemaker_training_role = get_cfn_output("NvidiaFraudDetectionTrainingRole", "SageMakerRoleArn")

training_repo = get_cfn_output("NvidiaFraudDetectionTrainingImageRepo", "TrainingImageRepo")

inference_host = get_inference_host()

print(f"Bucket Name: {bucket_name}")
print(f"Training Role: {sagemaker_training_role}")
print(f"Training Repo: {training_repo}")
print(f"Inference Host: {inference_host}")

----
# Step 1: Get and Prepare the data

## For Local
1. Download the dataset: https://ibm.ent.box.com/v/tabformer-data/folder/130747715605
2. untar and uncompreess the file: `tar -xvzf ./transactions.tgz`
3. Put card_transaction.v1.csv in in the `data/TabFormer/raw` folder


## For Brev 
1. Download the dataset: https://ibm.ent.box.com/v/tabformer-data/folder/130747715605
2. In the Jupyter notebook window, use the "File Browser" section to the data/Tabformer/raw folder
3. Drag-and-drop the "transactions.tgz" file into the folder
    - There is also an "upload" option that displays a file selector
    - Please wait for the upload to finish, it could take a while, by lookign at the status indocator at the bottom of the window
4. Now uncompress and untar by running the following command
    - Note: if somethign goes wrong you will need to delete the file rather than trying to overwrite it.

In [ ]:
# verify that the compressed file was uploaded successfully - the size should be 266M
!ls -lh ../data/TabFormer/raw

In [ ]:
# Uncompress/untar the file
!tar xvzf ../data/TabFormer/raw/transactions.tgz -C ../data/TabFormer/raw/

__If__ drag-and-drop is not working, please run the [Download TabFormer](./extra/download-tabformer.ipynb) notebook is the "extra" folder 

## Check data folder structure
The goal is to produce the following structure

```
.
    data
    └── TabFormer
        └── raw
            └── card_transaction.v1.csv
```

In [ ]:
# Once the raw data is placed as described above, set the path to the TabFormer directory

# Change this path to point to TabFormer data
data_root_dir = os.path.abspath('../data/TabFormer/') 
# Change this path to the directory where you want to save your model
model_output_dir = os.path.join(data_root_dir, 'trained_models')

# Path to save the trained model
os.makedirs(model_output_dir, exist_ok=True)

### Define python function to print directory tree

In [ ]:
def print_tree(directory, prefix=""):
    """Recursively prints the directory tree starting at 'directory'."""
    # Retrieve a sorted list of entries in the directory
    entries = sorted(os.listdir(directory))
    entries_count = len(entries)
    
    for index, entry in enumerate(entries):
        path = os.path.join(directory, entry)
        # Determine the branch connector
        if index == entries_count - 1:
            connector = "└── "
            extension = "    "
        else:
            connector = "├── "
            extension = "│   "
        
        print(prefix + connector + entry)
        
        # If the entry is a directory, recursively print its contents
        if os.path.isdir(path):
            print_tree(path, prefix + extension)

In [ ]:
# Check if the raw data has been placed properly
print_tree(data_root_dir)

---
# Step 2: Preprocess the data 
- Import the Python function for preprocessing the TabFormer data
- Call `preprocess_TabFormer` function to prepare the data

NOTE: The preprocessing can takes a few minutes


In [ ]:
# Add the "src" directory to the search path
src_dir = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), 'src'))
sys.path.insert(0, src_dir)

# should be able to import from "src" folder now
from preprocess_TabFormer_lp import preprocess_data

In [ ]:
# Preprocess the data
user_mask_map, mx_mask_map, tx_mask_map = preprocess_data(data_root_dir)

# this will output status as it correlates different attributes with target column

In [ ]:
print_tree(data_root_dir)

In [ ]:
# copy data to S3 to get pulled during training
! aws s3 sync "../data/TabFormer/" s3://$bucket_name/data/ --exclude "*/test_gnn" --force --delete

upload: ../data/TabFormer/gnn/nodes/offset_range_of_training_node.json to s3://ml-on-containers/data/gnn/nodes/offset_range_of_training_node.json
upload: ../data/TabFormer/gnn/nodes/node_label.csv to s3://ml-on-containers/data/gnn/nodes/node_label.csv
upload: ../data/TabFormer/gnn/edges/node_to_node.csv to s3://ml-on-containers/data/gnn/edges/node_to_node.csv
upload: ../data/TabFormer/gnn/test_gnn/edges/node_to_node.csv to s3://ml-on-containers/data/gnn/test_gnn/edges/node_to_node.csv
upload: ../data/TabFormer/gnn/test_gnn/nodes/node_label.csv to s3://ml-on-containers/data/gnn/test_gnn/nodes/node_label.csv
upload: ../data/TabFormer/xgb/test.csv to s3://ml-on-containers/data/xgb/test.csv
upload: ../data/TabFormer/gnn/nodes/node.csv to s3://ml-on-containers/data/gnn/nodes/node.csv
upload: ../data/TabFormer/xgb/validation.csv to s3://ml-on-containers/data/xgb/validation.csv
upload: ../data/TabFormer/gnn/test_gnn/nodes/node.csv to s3://ml-on-containers/data/gnn/test_gnn/nodes/node.csv
uplo

-----
# Step 3:  Now train the model using the financial-fraud-training container


## Create training configuration file
NOTE: Training configuration file must conform to schema defined in docs (to be updated.)

__Important: Models and configuration files needed for deployment using NVIDIA Dynamo-Triton will be saved in model-repository under the folder that is mounted in /trained_models inside the container__

In [ ]:
training_config = {
  "paths": {
        "data_dir": "/opt/ml/input/data/data",
        "output_dir": "/opt/ml/model"
  },

  "models": [
    {
      "kind": "GNN_XGBoost",
      "gpu": "single",
      "hyperparameters": {
        "gnn":{
          "hidden_channels": 32,
          "n_hops": 2,
          "layer": "SAGEConv",
          "dropout_prob": 0.1,
          "batch_size": 4096,
          "fan_out": 10,
          "num_epochs": 1
        },
        "xgb": {
          "max_depth": 6,
          "learning_rate": 0.2,
          "num_parallel_tree": 3,
          "num_boost_round": 512,
          "gamma": 0.0
        }

      }
    }
  ]
}


#### Save the training configuration as a json file

In [ ]:
training_config_file_name = 'training_config.json'

with open(os.path.join(training_config_file_name), 'w') as json_file:
    json.dump(training_config, json_file, indent=4)

# clone config to S3
! aws s3 cp ./training_config.json s3://$bucket_name/config/training_config.json

## Train model using financial_fraud_training container

In [ ]:
# run the training in Sagemaker
# define training job
training_job_name = "fraud-detection-gnn-" + str(int(time.time()))

# send training job to parameter store
ssm_client = boto3.client("ssm")

ssm_client.put_parameter(Name="/triton/model", Value=training_job_name, Overwrite=True)

sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.create_training_job(
    TrainingJobName=training_job_name,
    RoleArn=sagemaker_training_role,
    AlgorithmSpecification={
        'TrainingImage': f"{training_image}:latest",
        'TrainingInputMode': 'File',
        'ContainerEntrypoint': [
            'torchrun',
            '--standalone',
            '--nnodes=1',
            '--nproc_per_node=1',
            'main.py',
            '--config',
            "/opt/ml/input/data/config/training_config.json"
        ]
    },
    InputDataConfig=[
        {
            'ChannelName': 'data',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': f's3://{bucket_name}/data/gnn/',
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'ContentType': 'application/x-directory',
            'InputMode': 'File',
            'CompressionType': 'None'
        },
        {
            'ChannelName': 'config',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': f's3://{bucket_name}/config/',
                    'S3DataDistributionType': 'FullyReplicated',
                }
            },
            'ContentType': 'application/x-directory',
            'InputMode': 'File',
            'CompressionType': 'None'
        }
    ],
    OutputDataConfig={
        'S3OutputPath': f's3://{bucket_name}/output/'
    },
    ResourceConfig={
        'InstanceType': 'ml.g5.xlarge',
        'InstanceCount': 1,
        'VolumeSizeInGB': 30
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 86400
    }
)

#### Make sure that the training job succeeds
According to the training configuration file defined earlier, if the training runs successfully

In [ ]:
from IPython.display import clear_output
def poll_training_status(job_name):
    sagemaker_client = boto3.client('sagemaker')
    
    while True:
        response = sagemaker_client.describe_training_job(TrainingJobName=job_name)
        status = response['TrainingJobStatus']
        
        # Clear previous output
        clear_output(wait=True)
        
        # Print current status with timestamp
        print(f"Job: {job_name}")
        print(f"Status: {status}")
        print(f"Last checked: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        if status in ['Completed', 'Failed', 'Stopped']:
            break
            
        # Wait before next check
        time.sleep(30)
    
    return status

In [ ]:
final_status = poll_training_status(training_job_name)

Job: new-nim-1755615683
Status: Completed
Last checked: 2025-08-19 15:16:31


----
# Step 4:  Serve your python backend model using NVIDIA Dynamo-Triton
__!Important__: Change MODEL_REPO_PATH to point to `{model_output_dir}` / `python_backend_model_repository` if you used a different path in your training configuration file

#### Install NVIDIA Dynamo-Triton Client

In [ ]:
!pip install 'tritonclient[all]'

In [ ]:
import tritonclient.grpc as triton_grpc
import tritonclient.http as httpclient
from tritonclient import utils as triton_utils


##### Replace HOST with the actual URL where your NVIDIA Dynamo-Triton server is hosted.


In [ ]:
HOST = inference_host
HTTP_PORT = 8005
GRPC_PORT = 8006
METRICS_PORT = 8007

### Serve your models with NVIDIA Dynamo-Triton

With the infrastructure repo deployed, we have a Lambda function waiting for the training job to complete and for the models to be output to `s3://ml-on-containers-<accountnumber>/output` and then, they'll get extracted to a different bucket to be served by the inference host setup by our infrastructure.

### URLs for GRPC and HTTP request to the inference server

In [ ]:
client_grpc = triton_grpc.InferenceServerClient(url=f'{HOST}:{GRPC_PORT}')
client_http = httpclient.InferenceServerClient(url=f'{HOST}:{HTTP_PORT}')

### Wait for NVIDIA Dynamo-Triton to install packages and come online
**NOTE**: This cell can take a few minutes to execute.
 If the following cell keeps running even after you see `Started HTTPService at {HOST}:{HTTP_PORT}` in the log, you can interrupt the execution of this cell and continue from the next cell.

In [ ]:
import subprocess

while True:
    client_grpc = triton_grpc.InferenceServerClient(url=f'{HOST}:{GRPC_PORT}')
    try:
        if client_grpc.is_server_ready():
            break
    except triton_utils.InferenceServerException as e:
        pass
    try:
        # Run the docker logs command with the --tail option
        output = "TODO: Figure out what's going on here and how to display container logs from k8s inference host"
        print(output.decode("utf-8"))
    except subprocess.CalledProcessError as e:
        print("Error retrieving logs:", e)
    time.sleep(10)

### Check if NVIDIA Dynamo-Triton is running properly

In [ ]:
#TODO: Figure out how to demonstrate that the pod is online from our container

#### Here’s an example of how to prepare data for inference, using random data

In [ ]:
import numpy as np

from tritonclient.http import InferenceServerClient, InferInput, InferRequestedOutput

def make_example_request():
    # -- example sizes --
    num_merchants = 5
    num_users   = 7
    num_edges   = 3
    merchant_feature_dim = 24
    user_feature_dim = 13
    user_to_merchant_feature_dim = 38

    # -- 1) features --
    x_merchant = np.random.randn(num_merchants, merchant_feature_dim).astype(np.float32)
    x_user   = np.random.randn(num_users, user_feature_dim).astype(np.float32)

    # -- 2) shap flag and masks --
    compute_shap          = np.array([True], dtype=np.bool_)
    feature_mask_merchant   = np.random.randint(0,2, size=(merchant_feature_dim,), dtype=np.int32)
    feature_mask_user     = np.random.randint(0,2, size=(user_feature_dim,), dtype=np.int32)

    # -- 3) edges: index [2, num_edges] and attributes [num_edges,user_to_merchant_feature_dim] --
    edge_index_user_to_merchant = np.vstack([
        np.random.randint(0, num_users,   size=(num_edges,)),
        np.random.randint(0, num_merchants, size=(num_edges,))
    ]).astype(np.int64)
    
    edge_attr_user_to_merchant = np.random.randn(num_edges, user_to_merchant_feature_dim).astype(np.float32)

    feature_mask_user_to_merchant =  np.random.randint(0,2, size=(user_to_merchant_feature_dim,), dtype=np.int32)

    return {
        "x_merchant": x_merchant,
        "x_user": x_user,
        "COMPUTE_SHAP": compute_shap,
        "feature_mask_merchant": feature_mask_merchant,
        "feature_mask_user": feature_mask_user,
        "edge_index_user_to_merchant": edge_index_user_to_merchant,
        "edge_attr_user_to_merchant": edge_attr_user_to_merchant,
        "edge_feature_mask_user_to_merchant": feature_mask_user_to_merchant
    }



In [ ]:
def prepare_and_send_inference_request(data):

    # Connect to Triton
    client = httpclient.InferenceServerClient(url=f'{HOST}:{HTTP_PORT}')

    # Prepare Inputs

    inputs = []
    def _add_input(name, arr, dtype):
        inp = InferInput(name, arr.shape, datatype=dtype)
        inp.set_data_from_numpy(arr)
        inputs.append(inp)

    for key, value in data.items():
        if key.startswith("x_"):
            dtype = "FP32"
        elif key.startswith("feature_mask_"):
            dtype = "INT32"
        elif key.startswith("edge_feature_mask_"):
            dtype = "INT32"            
        elif key.startswith("edge_index_"):
            dtype = "INT64"
        elif key.startswith("edge_attr_"):
            dtype = "FP32"
        elif key == "COMPUTE_SHAP":
            dtype = "BOOL"
        else:
            continue  # skip things we don't care about

        _add_input(key, value, dtype)


    # Outputs

    outputs = [InferRequestedOutput("PREDICTION")]

    for key in data:
        if key.startswith("x_"):
            node = key[len("x_"):]  # extract node name
            outputs.append(InferRequestedOutput(f"shap_values_{node}"))
        elif key.startswith("edge_attr_"):
            edge_name = key[len("edge_attr_"):]  # extract edge name
            outputs.append(InferRequestedOutput(f"shap_values_{edge_name}"))
    
    # Send request

    model_name="prediction_and_shapley"
    response = client.infer(
        model_name,
        inputs=inputs,
        request_id=str(1),
        outputs=outputs,
        timeout= 3000
    )

    result = {}

    # always include prediction
    result["PREDICTION"] = response.as_numpy("PREDICTION")

    # add shap values
    for key in data:
        if key.startswith("x_"):
            node = key[len("x_"):]  # e.g. "merchant", "user"
            result[f"shap_values_{node}"] = response.as_numpy(f"shap_values_{node}")
        if key.startswith("edge_attr_"):
            edge_name = key[len("edge_attr_"):]  # e.g. ("user" "to"  "merchant")
            result[f"shap_values_{edge_name}"] = response.as_numpy(f"shap_values_{edge_name}")
    
    return result


## Prediction without computing Shapley values

### Read preprocessed input transactions to send query to NVIDIA Dynamo-Triton

In [ ]:
import os
import pandas as pd
import numpy as np

def load_hetero_graph(gnn_data_dir):
    """
    Reads:
      - All node CSVs from nodes/, plus their matching feature masks (<node>_feature_mask.csv)
        If missing, a mask of all ones is created (np.int32).
      - All edge CSVs from edges/:
          base        -> edge_index_<edge> (np.int64)
          *_attr.csv  -> edge_attr_<edge>  (np.float32)
          *_label.csv -> exactly one -> edge_label_<edge> (DataFrame)
    """
    base = os.path.join(gnn_data_dir, "test_gnn")
    nodes_dir = os.path.join(base, "nodes")
    edges_dir = os.path.join(base, "edges")

    out = {}
    node_feature_mask = {}

    # --- Nodes: every CSV becomes x_<node>; also read/create feature_mask_<node> ---
    if os.path.isdir(nodes_dir):
        for fname in os.listdir(nodes_dir):
            if fname.lower().endswith(".csv") and not fname.lower().endswith("_feature_mask.csv"):
                node_name = fname[:-len(".csv")]
                node_path = os.path.join(nodes_dir, fname)
                node_df = pd.read_csv(node_path)
                out[f"x_{node_name}"] = node_df.to_numpy(dtype=np.float32)

                # feature mask file (optional)
                mask_fname = f"{node_name}_feature_mask.csv"
                mask_path = os.path.join(nodes_dir, mask_fname)
                if os.path.exists(mask_path):
                    mask_df = pd.read_csv(mask_path, header=None)
                    node_feature_mask[node_name] = mask_df
                    feature_mask = mask_df.to_numpy(dtype=np.int32).ravel()
                else:
                    # create a must with all zeros
                    feature_mask = np.zeros(node_df.shape[1], dtype=np.int32)
                out[f"feature_mask_{node_name}"] = feature_mask

    # --- Edges: group into base, attr, label by filename suffix ---
    base_edges = {}
    edge_attrs = {}
    edge_labels = {}
    edge_feature_mask = {}

    if os.path.isdir(edges_dir):
        for fname in os.listdir(edges_dir):
            if not fname.lower().endswith(".csv"):
                continue
            path = os.path.join(edges_dir, fname)
            lower = fname.lower()
            if lower.endswith("_attr.csv"):
                edge_name = fname[:-len("_attr.csv")]
                edge_attrs[edge_name] = pd.read_csv(path) #, header=None)
            elif lower.endswith("_label.csv"):
                edge_name = fname[:-len("_label.csv")]
                edge_labels[edge_name] = pd.read_csv(path)
            elif lower.endswith("_feature_mask.csv"):
                edge_name = fname[:-len("_feature_mask.csv")]
                edge_feature_mask[edge_name] = pd.read_csv(path, header=None)
            else:
                edge_name = fname[:-len(".csv")]
                base_edges[edge_name] = pd.read_csv(path) #, header=None)



    # Enforce: only one label file total
    if len(edge_labels) == 0:
        raise FileNotFoundError("No '*_label.csv' found in edges/. Exactly one label file is required.")
    if len(edge_labels) > 1:
        raise ValueError(f"Found multiple label files: {list(edge_labels.keys())}. Exactly one is allowed.")

    # Build output keys for edges
    for edge_name, df in base_edges.items():
        out[f"edge_index_{edge_name}"] = df.to_numpy(dtype=np.int64).T
        if edge_name in edge_attrs:
            out[f"edge_attr_{edge_name}"] = edge_attrs[edge_name].to_numpy(dtype=np.float32)
        if edge_name in edge_feature_mask:
            out[f"edge_feature_mask_{edge_name}"] = edge_feature_mask[edge_name].to_numpy(dtype=np.int32).ravel()
        else:
            # create a must with all zeros
            out[f"edge_feature_mask_{edge_name}"] = np.zeros(edge_attrs[edge_name].shape[1], dtype=np.int32)

        

    # Add the single label file (kept as DataFrame)
    (label_edge_name, label_df), = edge_labels.items()
    out[f"edge_label_{label_edge_name}"] = label_df

    return out

In [ ]:
test_data = load_hetero_graph(gnn_data_dir)
compute_shap = False
result =  prepare_and_send_inference_request(test_data | {"COMPUTE_SHAP": np.array([compute_shap], dtype=np.bool_)})

In [ ]:
result['PREDICTION']

### Evaluate performance on test data

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score)
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:

def compute_score_for_batch(y, predictions, decision_threshold = 0.5):
    # Apply threshold
    y_pred = (predictions > decision_threshold).astype(int)

    # Compute evaluation metrics
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, zero_division=0)
    recall = recall_score(y, y_pred, zero_division=0)
    f1 = f1_score(y, y_pred, zero_division=0)

    # Confusion matrix
    classes = ['Non-Fraud', 'Fraud']
    columns = pd.MultiIndex.from_product([["Predicted"], classes])
    index = pd.MultiIndex.from_product([["Actual"], classes])

    conf_mat = confusion_matrix(y, y_pred)
    cm_df = pd.DataFrame(conf_mat, index=index, columns=columns)
    print(cm_df)

    # Plot the confusion matrix directly
    disp = ConfusionMatrixDisplay.from_predictions(
        y, y_pred, display_labels=classes
    )
    disp.ax_.set_title('Confusion Matrix')
    plt.show()

    # Print summary
    print("----Summary---")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")


#### Scores on test data

In [ ]:
# Decision threshold to flag a transaction as fraud
#Change to trade-off precision and recall
decision_threshold = 0.5
y = test_data['edge_label_user_to_merchant'].to_numpy(dtype=np.int32)
compute_score_for_batch(y, result['PREDICTION'], decision_threshold)

### Compute Shapley values of different features for a transaction
NOTE: Shapely computation is very expensive, it will only compute shap values for first transaction.

#### Shapley values for different features

In [ ]:
test_data =  load_hetero_graph(gnn_data_dir)
compute_shap = True
result_with_shap = prepare_and_send_inference_request(test_data | {"COMPUTE_SHAP": np.array([compute_shap], dtype=np.bool_)})

In [ ]:
for key in result_with_shap:
    if key.startswith('shap_'):
        print(f'{key} : {result_with_shap[key]}')

In [ ]:
feature_masks = {
    'user': user_mask_map,
    'merchant': mx_mask_map,
    'user_to_merchant': tx_mask_map
}

In [ ]:
for key in feature_masks:
    shap_values = result_with_shap[f'shap_values_{key}']
    min_idx = min(feature_masks[key].values())

    attr_to_shap = {
        attr: float(shap_values[int(idx - min_idx)])
        for attr, idx in feature_masks[key].items()
    }
    print(attr_to_shap)